# **Semantic Spotter Project Submission by Satya Prakash - C58 Batch**

# Option 1: Build a RAG System - RAG Based Chatbot implemented using Langchain to search information in Life Insurance Document.
## Overview
This project builds a Retrieval-Augmented Generation (RAG) chatbot using LangChain to effectively answer user queries baseThis project builds a Retrieval-Augmented Generation (RAG) chatbot using LangChain to automate the process of extracting and answering questions based on life insurance policies. The system combines document processing, vector-based retrieval, caching, and generative AI to efficiently generate concise answers. LangChain simplifies the integration of these components by providing a modular framework to connect document loaders, vector databases, language models, and custom chains.

The chatbot extracts information from policy documents, stores it in ChromaDB (a vector store), and uses OpenAI’s GPT-3.5-turbo to generate answers. With LangChain’s retrieval and caching mechanisms, the system can optimize responses for both complex queries and repeated questions.

## Problem Statement
Insurance documents are long, complex, and challenging to navigate. Customers or employees seeking specific policy details (such as benefits or claims procedures) often struggle to find the relevant sections quickly. Searching through large PDFs manually is time-consuming and prone to errors, leading to frustration for both internal staff and customers.

The goal of this project is to automate the retrieval and question-answering process using RAG-based techniques. The system needs to:

- Extract relevant sections from insurance documents.
- Provide accurate and concise answers to user queries.
- Optimize response time by leveraging a caching mechanism for repeated queries.
- Use generative AI (GPT-3.5) to answer complex, context-dependent questions.
- Ensure that answers are well-cited, linking back to specific sections or pages from the source documents.

## Architecture
The following is the design of the system:

1. PDF Processing and Document Chunking:
  - Use LangChain’s PyPDFLoader to extract text from insurance policy PDFs.
  - Chunk large documents into manageable pieces to ensure relevant information is indexed correctly.
2. Embedding and Vector Store Setup:
  - Generate text embeddings using OpenAI’s text-embedding-ada-002 model.
  - Store these embeddings in a persistent Chroma vector database for efficient retrieval.
3. Retrieval and Caching:
  - Implement a retriever that searches for relevant sections based on user queries using similarity search.
  - Incorporate a cache mechanism to store frequent queries and minimize redundant computation.
4. Prompt Engineering and Response Generation:
  - Use LLMChain to connect the retrieved information with a structured prompt for GPT-3.5-turbo.
  - Ensure the generated responses are concise, informative, and cited, including references to specific policy names and pages.
5. User-Friendly Query Handling:
  - Provide clear and actionable answers to user questions (e.g., "What are the policy benefits for accidental death?").
  - Support both first-time queries and cached queries to enhance the user experience.

## 1. Install and Import the Required Libraries

In [1]:
# Install all the required libraries

!pip install langchain openai pdfplumber sentence-transformers tiktoken pypdf langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0

In [2]:
!pip install chromaDB==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.9 MB/s eta 0:00:

In [3]:
# Import all the required Libraries

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from sentence_transformers import CrossEncoder
import os

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
os.chdir('/content/drive/MyDrive/Colab_Notebooks/Semantic_Spotter_Project')
!ls

OPENAI_API_Key.txt			    Semantic_Spotter_RAG_Langchain_Satya_Prakash.ipynb
Principal-Sample-Life-Insurance-Policy.pdf


In [6]:
# Set OpenAI API key
with open("/content/drive/MyDrive/Colab_Notebooks/Semantic_Spotter_Project/OPENAI_API_Key.txt", "r") as f:
    openai_api_key = ' '.join(f.readlines())

## 2. Read, Process, and Chunk the PDF File

In [7]:
# Load and process PDFs using LangChain's PyPDFLoader
pdf_directory = "/content/drive/MyDrive/Colab_Notebooks/Semantic_Spotter_Project/"
pdf_loader = PyPDFLoader(pdf_directory + "Principal-Sample-Life-Insurance-Policy.pdf")

In [8]:
# Extract documents from the PDF
documents = pdf_loader.load()

In [9]:
# Use RecursiveCharacterTextSplitter to chunk the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [10]:
texts

[Document(metadata={'source': '/content/drive/MyDrive/Colab_Notebooks/Semantic_Spotter_Project/Principal-Sample-Life-Insurance-Policy.pdf', 'page': 0}, page_content='GROUP POLICY FOR:  \nRHODE ISLAND JOHN DOE  \n \nALL MEMBERS  \nGroup Member Life Insurance  \n \nPrint Date: 07/16/2014  \n DOROTHEA GLAUSE  S655  \nRHODE ISLAND JOHN DOE  01/01/2014  \n711 HIGH STREET   \nGEORGE RI 02903'),
 Document(metadata={'source': '/content/drive/MyDrive/Colab_Notebooks/Semantic_Spotter_Project/Principal-Sample-Life-Insurance-Policy.pdf', 'page': 1}, page_content='This page left blank intentionally'),
 Document(metadata={'source': '/content/drive/MyDrive/Colab_Notebooks/Semantic_Spotter_Project/Principal-Sample-Life-Insurance-Policy.pdf', 'page': 2}, page_content='GC 806 VAL   \n POLICY RIDER  \n \nGROUP INSURANCE      \n \n \nPOLICY NO:  S655  \n \nCOVERAGE:  Life  \n EMPLOYER: RHODE ISLAND JOHN DOE   \n Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the followi

## 3. Generate and Store Embeddings using OpenAI and ChromaDB

In this section, we will embed the pages in the dataframe through OpenAI's `text-embedding-ada-002` model, and store them in a ChromaDB collection.

In [11]:
# Set up OpenAI embeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)

<ipython-input-11-13946b8a9bdc>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)


In [12]:
# Initialize Chroma for vector storage with persistent storage path
chroma_persist_path = "/content/drive/MyDrive/Colab_Notebooks/Semantic_Spotter_Project/ChromaDB_Data"
vector_store = Chroma.from_documents(texts, embedding_model, persist_directory=chroma_persist_path)

In [13]:
# Save the vector store for future use
vector_store.persist()

<ipython-input-13-0b60a2cb076c>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [14]:
# Implement caching using LangChain's VectorStore
cache_store = Chroma(persist_directory=chroma_persist_path, embedding_function=embedding_model)

<ipython-input-14-04e0ee63cd64>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  cache_store = Chroma(persist_directory=chroma_persist_path, embedding_function=embedding_model)


## 4. Semantic Search with Cache

In this section, we will perform a semantic search of a query in the collections embeddings to get several top semantically similar results.

In [15]:
# Define a retrieval-based question-answering chain with LangChain
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [16]:
# Define query handling with caching
def get_results_with_cache(query: str):
    # Search in cache first
    cache_results = cache_store.similarity_search(query, k=1)

    # Check if cache_results is not empty and if the first element has 'score' before accessing it
    if cache_results and hasattr(cache_results[0], 'score') and cache_results[0].score < 0.2:  # Check if 'score' attribute exists
        print("Found in cache!")
        return cache_results
    else:
        print("Not found in cache. Searching main collection...")
        results = retriever.get_relevant_documents(query)
        cache_store.add_texts([query], metadatas=[{"query": query}])
        return results

In [17]:
# Define a prompt template for LangChain
prompt_template = """
You are a helpful assistant specializing in insurance policies. Answer the user's query accurately using the provided documents.
Query: {query}
Documents: {context}

Respond clearly and concisely, citing relevant policy names and page numbers where appropriate.
"""

prompt = PromptTemplate(input_variables=["query", "context"], template=prompt_template)

In [18]:
# Perform a sample query
query = "What are the policy benefits for accidental death?"

In [19]:
# Read the user query

#query = input()

In [20]:
# Get relevant documents (with caching support)
results = get_results_with_cache(query)

Not found in cache. Searching main collection...


<ipython-input-16-a28314e26050>:12: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [21]:
results

[Document(metadata={'page': 54, 'source': '/content/drive/MyDrive/Colab_Notebooks/Semantic_Spotter_Project/Principal-Sample-Life-Insurance-Policy.pdf'}, page_content='This policy has been updated effective  January 1, 2014  \n \n      PART IV - BENEFITS  \nGC 6015  Section B - Member Accidental Death and \nDismemberment Insurance, Page 3  \n  \nExposure  \n \nExposure to the elements will be presumed to be an injury if:  \n \na. such exposure is due to an accidental bodily injury; and  \n b. within 365 days after the injury, the Member incurs a loss that is the result of the exposure; and \n c. this Group Policy would have covered the injury resulting from the accident. \n  \nArticle 4 - Seat Belt/Airbag Benefit   \n \nIf the Member loses his or her life as a result of an acciden tal injury sustained while driving or \nriding in an Automobile, an additional benefit of $10,000 will be paid to the beneficiary named \nfor Member Life Insurance, provided all Benefit Qualifications as descr

In [22]:
# Format the retrieved documents into a context string
context = "\n\n".join([doc.page_content for doc in results])

## 4. Generation LLM output using the GPT-3.5-turbo Model

In this section, we will generate the output using langchain chains

In [23]:
# Initialize the OpenAI chat model
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)

<ipython-input-23-8f3501ee7209>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)


In [24]:
# Define an LLMChain to structure the interaction between the LLM and the prompt
llm_chain = LLMChain(prompt=prompt, llm=llm)

<ipython-input-24-100b8634839a>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


In [25]:
# Prepare the inputs for the LLMChain
inputs = {"query": query, "context": context}

In [26]:
# Generate the final response using the LLMChain
response = llm_chain.run(inputs)

<ipython-input-26-89e7d16e28ca>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run(inputs)


In [27]:
import textwrap

# wraping response to a width of 150 characters
wrapped_response = textwrap.fill(response, width=150)
print(wrapped_response)

The policy benefits for accidental death under Section B - Member Accidental Death and Dismemberment Insurance include the following:  - Scheduled
Benefit of $10,000 for loss of life (Section B, Page 1) - Additional $10,000 benefit for loss of life in Automobile accidents with seat belt and
airbag (Article 4) - Various percentages of the Scheduled Benefit for specific losses like paralysis, loss of limbs, speech, and hearing (Page 4, Page
5) - Repatriation Benefit of up to $2,000 for transportation of the body (Article 7) - Educational Benefit of $3,000 annually for a maximum of four
years for a Qualified Student (Article 8)  Please refer to Section B of the policy document, specifically Pages 1, 4, and 5 for detailed information
on benefits payable for accidental death.
